<a href="https://colab.research.google.com/github/AmrRagab0/Car-Plate-Detection-Recognition/blob/master/training_yolov5_on_plates_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Amr

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install gwpy --quiet

     |████████████████████████████████| 1.4 MB 6.8 MB/s 
     |████████████████████████████████| 51 kB 8.2 MB/s 
     |████████████████████████████████| 45 kB 3.7 MB/s 
     |████████████████████████████████| 11.2 MB 60.4 MB/s 
     |████████████████████████████████| 4.1 MB 56.3 MB/s 
     |████████████████████████████████| 957 kB 74.0 MB/s 


# FiftyOne

In [ ]:
%%capture
!pip install opencv-python-headless==4.5.4.60


In [ ]:
%%capture
!pip install fiftyone


In [ ]:
import fiftyone as fo


Migrating database to v0.16.6


INFO:fiftyone.migrations.runner:Migrating database to v0.16.6


# Preparing the dataset

In [ ]:
!cp -R /content/gdrive/MyDrive/Internship/yollo /content/yollo # copy content of yollo to colab


In [ ]:
import os
import shutil
count = 0
source_folder = "/content/gdrive/MyDrive/Internship_car/images/test/data/"
destination_folder = "/content/images/test/data/"

for file_name in os.listdir(source_folder):
    count += 1
    source = source_folder + file_name
    destination = destination_folder + file_name
    shutil.copy(source, destination)
    #print('copied', file_name)
print(count)

453


In [ ]:
name = "dataset"
dataset_dir = "/content/gdrive/MyDrive/Internship_car/datasetCOCO"

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.COCODetectionDataset,
    name=name,
)

# View summary info about the dataset
print(dataset)

# Print the first few samples in the dataset
print(dataset.head())

### Splitting the data

In [ ]:
export_dir = "/content/gdrive/MyDrive/Internship/yollo"
label_field = "ground_truth"  # for example

# The splits to export
splits = ["train", "val","test"]

# All splits must use the same classes list
classes=['standard','non-standard','car', 'motorcycle', 'airplane', 'bus', 'train', 'truck']

# Export the splits
for split in splits:
    split_view = dataset.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split=split,
        classes=classes,
    )

NameError: ignored

In [ ]:
#print(dataset.count_sample_tags())
%%capture
!pip install opencv-python-headless==4.5.4.60
!pip install fiftyone

In [ ]:
#!pip install fiftyone.utils.splits
#import fiftyone.utils.splits as fous
#fous.random_split(dataset, {"train": 0.7, "test": 0.2, "val": 0.1})


#YOLOV5

### Setting up Yolo

In [ ]:
%%capture
!pip install wandb
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

In [ ]:
%%capture
import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

copying images from original drive to my drive ( to avoid the error)

In [ ]:
!cp -R /content/gdrive/MyDrive/Internship_car/yollo/images /content/yollo


In [ ]:
!cp -R /content/gdrive/MyDrive/car_annotions/All/labels /content/yollo

# Training

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data /content/yollo/dataset.yaml --cfg yolov5x.yaml --weights yolov5x.pt --cache --hyp hyp.scratch-high.yaml
#                  img size
#!cp -R /content/yolov5 /content/gdrive/MyDrive/Internship_car/yollo



wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 3
wandb: You chose 'Don't visualize my results'
train: weights=yolov5x.pt, cfg=yolov5x.yaml, data=/content/yollo/dataset.yaml, hyp=hyp.scratch-high.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-77-g4a37381 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, mome

Note :  Always train from a local dataset. Mounted or network drives like Google Drive will be very slow

In [ ]:
!cp -r "/content/yolov5/runs/train/exp/weights/best.pt" "/content/gdrive/MyDrive/Internship_car/Weights/"

In [ ]:
33# we can also output some older school graphs if the tensor board isn't working for whatever reason...
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/exp/results.png', width=1000)  # view results.png

FileNotFoundError: ignored

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/exp/val_batch0_labels.jpg', width=900)

In [ ]:
# Test
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 416 --conf 0.4 --source /content/yollo/images/test

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp3/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 416 --conf 0.4 --source /content/gdrive/MyDrive/Internship_car/testC

In [ ]:
!cp -R /content/yolov5/runs/detect/exp /content/drive/MyDrive/Internship/yollo


*** Can see Output : https://drive.google.com/drive/folders/1LW0Iikm_YwZQS3IUZ_V3qeq9F4GSbsqw?usp=sharing ***

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.png'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 416 --data /content/yollo/dataset.yaml --task test --name yolo_test


# Exporting the model

In [ ]:
!python export.py --weights yolov5s.pt

In [ ]:
from google.colab import drive
drive.flush_and_unmount()